# **Implémentation from scratch de Word2Vec**

Le projet porte sur l'implémentation, from sratch, des différentes variantes de Word2Vec (CBOW et Skim-gram). Cela vous permettra de comprendre en profondeur les mécanismes derrière les embeddings de mots, en partant des concepts fondamentaux jusqu'aux variantes améliorées.

## **A. Structure du projet**

### 1. Introduction et fondements théoriques
- Explication des concepts fondamentaux de représentation vectorielle des mots
- Histoire et évolution des embeddings de mots
- Principes mathématiques sous-jacents (calcul de probabilités, descente de gradient, etc.)

### 2. Préparation des données
- Création d'un pipeline de prétraitement textuel
  - Normalisation (mise en minuscule, suppression des caractères spéciaux)
  - Tokenisation
  - Gestion des mots rares (filtrage par fréquence)
- Construction du vocabulaire et mapping mot-indice
- Génération des paires d'apprentissage (contexte/cible)
- Implémentation de techniques d'échantillonnage négatif et de sous-échantillonnage de mots fréquents

### 3. Implémentation du modèle CBOW (Continuous Bag of Words)
- Architecture du réseau: entrée → projection → sortie
- Implémentation de la fonction de perte
- Calcul du gradient et mise à jour des poids
- Optimisation avec différentes méthodes (SGD, Adam, etc.)
- Évaluation des performances sur des tâches simples

### 4. Implémentation du modèle Skip-gram
- Architecture du modèle inverse au CBOW
- Comparaison des performances avec CBOW
- Analyse des avantages pour les mots rares

### 5. Optimisations et variantes avancées
- Negative sampling (échantillonnage négatif)
- Subsampling of frequent words (sous-échantillonnage des mots fréquents)
- Comparaison de l'efficacité computationnelle et qualité des embeddings

### 6. Visualisation et évaluation
- Réduction de dimensionnalité pour visualisation (t-SNE, PCA)
- Évaluation sur des tâches d'analogie ("roi - homme + femme = reine")
- Mesure de similarité cosinus entre mots

### 7. Application pratique
- Utilisation des embeddings dans une tâche de NLP simple (classification de texte, analyse de sentiment)
- Analyse de l'impact de différentes variantes sur la performance finale
- Comparaison avec des embeddings pré-entraînés: Word2Vec de Google


## Implémentation progressive

Pour l'implémentation du projet, procéder par une approche progressivecomme suit :

1. **Phase 1**: Implémentation des versions simplifiées
   - Commencez par implémenter les versions basiques de CBOW et Skip-gram sans optimisations
   - Utilisez un petit corpus pour tester rapidement
   - Visualisez les premiers résultats pour vérifier que l'apprentissage fonctionne

2. **Phase 2**: Ajout des optimisations
   - Intégrez l'échantillonnage négatif
   - Ajoutez le sous-échantillonnage des mots fréquents
   - Mesurez l'impact sur les performances et la vitesse d'entraînement

3. **Phase 3**: Évaluation et applications
   - Évaluez systématiquement les différentes variantes
   - Analysez les représentations obtenues
   - Utilisez les embeddings dans une application concrète


## **B. Bases de données**


### 1.[20 Newsgroups](https://www.kaggle.com/datasets/crawford/20-newsgroups)
La base de données contient environ 20,000 documents provenant de 20 groupes de discussion différents. Chaque message appartient à une catégorie spécifique, ce qui en fait un excellent candidat pour la classification multiclasse. Ce corpus est particulièrement intéressant car il contient du langage naturel avec des variations stylistiques importantes, ce qui met à l'épreuve la robustesse des embeddings.

### 2. [IMDb Movie Reviews](https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews)
Une base de données de 50,000 critiques de films avec des étiquettes de sentiment (positif/négatif). C'est une base intéressante pour tester les embeddings dans une tâche de classification binaire de sentiment. Ce corpus est particulièrement utile pour évaluer si vos embeddings capturent bien les nuances émotionnelles et contextuelles.

## Méthodologie de test recommandée

Pour évaluer efficacement vos embeddings Word2Vec, je vous conseille de:

1. **Diviser votre travail en deux phases distinctes**:
   - Phase d'entraînement des embeddings sur l'intégralité du corpus textuel (sans utiliser les étiquettes)
   - Phase de classification utilisant les embeddings comme features

2. **Comparer différentes représentations**:
   - Embeddings CBOW vs Skip-gram
   - Différentes dimensions d'embeddings (100, 200, 300)
   - Différentes tailles de fenêtre contextuelle
   - Avec vs sans les optimisations (negative sampling, Subsampling of Frequent Words)

3. **Évaluer l'impact des prétraitements**:
   - Lemmatisation vs stemming vs tokens bruts
   - Avec vs sans suppression des mots vides (stopwords)
   - Avec vs sans normalisation des chiffres/entités nommées

Pour la phase de classification proprement dite, utilisez un classifieur simple (un ANN) pour isoler l'effet des embeddings sur les performances. Vous pourriez représenter chaque document par la moyenne des vecteurs de tous les mots du document.

## Conseil pratique

Si vous manquez de ressources computationnelles, commencez par une fraction du corpus (10-20%) pour l'entraînement des embeddings et augmentez progressivement. Cela vous permettra de détecter rapidement les problèmes potentiels sans attendre des jours pour l'entraînement complet.

Ces méthodologies vous permettront non seulement d'évaluer la performance de vos implémentations Word2Vec, mais aussi de comprendre quelles variantes fonctionnent le mieux pour différents types de classification textuelle.

## **C. Techniques d'Optimisation pour Word2Vec**

## **1. Negative Sampling**

Cette technique d'optimisations vise à résoudre un problème fondamental : la complexité computationnelle excessive lors de l'entraînement des modèles d'embeddings.

## Le problème à résoudre

Dans les modèles Word2Vec classiques (CBOW et Skip-gram), pour chaque prédiction, le modèle doit calculer une probabilité pour chaque mot du vocabulaire. Pour un vocabulaire de taille V (qui peut facilement atteindre 100,000 mots ou plus), cela signifie calculer V probabilités pour chaque exemple d'entraînement, puis mettre à jour les poids correspondants. Cette opération devient extrêmement coûteuse en calcul et en mémoire.

## Negative Sampling (Échantillonnage négatif)

### Principe fondamental

L'échantillonnage négatif transforme le problème de classification multi-classes (parmi V classes) en plusieurs problèmes de classification binaire.

Au lieu de prédire "quel est le mot cible parmi tous les mots du vocabulaire", le modèle répond à une question plus simple : "ce mot spécifique est-il le mot cible (positif) ou non (négatif) ?"

### Fonctionnement détaillé

1. Pour chaque exemple d'apprentissage (mot contexte → mot cible), le modèle apprend à:
   - Reconnaître le mot cible réel comme positif
   - Reconnaître K mots "négatifs" (échantillonnés aléatoirement) comme négatifs

2. La fonction objective devient une série de problèmes logistiques binaires:
   - Maximiser la probabilité que le vrai mot cible soit positif
   - Maximiser la probabilité que les K mots échantillonnés soient négatifs

3. La stratégie d'échantillonnage est importante. Les mots sont échantillonnés selon une distribution basée sur leur fréquence, généralement:

   P(w_i) = f(w_i)^(3/4) / Σ f(w_j)^(3/4)

   où f(w_i) est la fréquence du mot w_i dans le corpus. L'exposant 3/4 est empirique et permet de surreprésenter légèrement les mots moins fréquents.

### Avantages

- La complexité passe de O(V) à O(K), où K est typiquement entre 5 et 20 (bien inférieur à V)
- Meilleure qualité des embeddings pour les mots rares
- Bon équilibre entre vitesse et qualité des représentations

### Exemple concret

Supposons que nous travaillons avec la phrase "le chat dort sur le canapé" et que nous voulons prédire "dort" à partir du contexte "le chat ___ sur le".

Avec negative sampling (K=2):
1. Exemple positif: (contexte → "dort") devrait donner une probabilité élevée
2. Exemples négatifs: (contexte → "voiture") et (contexte → "mange") devraient donner des probabilités faibles

Au lieu de calculer les probabilités pour tous les mots du vocabulaire, nous n'en calculons que 3.


## Implémentation dans votre projet

   - Implémentez une distribution d'échantillonnage basée sur la fréquence
   - Testez différentes valeurs de K (5, 10, 15, 20)
   - Mesurez l'impact sur la vitesse d'entraînement et la qualité des embeddings


## **2. Subsampling of Frequent Words (Sous-échantillonnage des mots fréquents)**

Le sous-échantillonnage des mots fréquents est une technique d'optimisation importante dans Word2Vec qui complète le negative sampling. Cette méthode aborde un problème fondamental dans l'apprentissage des embeddings : la surreprésentation des mots très fréquents et son impact sur la qualité des représentations vectorielles.

## Problème résolu par le sous-échantillonnage

Dans un texte naturel, certains mots comme "le", "la", "et", "de", "un" apparaissent avec une fréquence extrêmement élevée. Ces mots, souvent appelés "mots vides" (stopwords), présentent plusieurs inconvénients dans l'entraînement de Word2Vec :

1. **Déséquilibre Informatif**
   - Les mots très fréquents (stopwords) apportent peu de valeur sémantique
   - Ils monopolisent l'attention du modèle d'apprentissage
   - Ils peuvent dégrader la qualité des représentations vectorielles

2. **Complexité Computationnelle**
   - Chaque occurrence de ces mots génère des calculs redondants
   - Ralentissement significatif de l'entraînement
   - Gaspillage de ressources computationnelles

3. **Biais dans la Représentation**
   - Les embeddings risquent d'être tirés vers des représentations génériques
   - Perte des nuances sémantiques pour les mots moins fréquents

## Principe du sous-échantillonnage

Le sous-échantillonnage consiste à écarter aléatoirement certaines occurrences des mots très fréquents pendant la phase de préparation des données d'entraînement. Plus un mot est fréquent, plus la probabilité qu'il soit écarté est élevée.

Concrètement, pour chaque mot w rencontré dans le texte, on décide de le conserver ou de l'écarter selon une probabilité P(w) définie par la formule suivante :

P(w) = (√(t/f(w)) + 1) × (t/f(w))

Où :
- f(w) est la fréquence relative du mot w dans le corpus (nombre d'occurrences divisé par la taille totale du corpus)
- t est un seuil, généralement fixé autour de 10^-5

Cette formule a plusieurs propriétés intéressantes :
- Pour les mots très fréquents (f(w) >> t), la probabilité de conservation devient très faible
- Pour les mots rares (f(w) << t), la probabilité de conservation est proche de 1
- La transition entre ces deux extrêmes est progressive

## Exemple concret

Imaginons un corpus où le mot "le" apparaît avec une fréquence relative de 5% (f("le") = 0.05) et le seuil t = 10^-5.

La probabilité de conserver "le" serait :
P("le") = (√(10^-5/0.05) + 1) × (10^-5/0.05) ≈ 0.0447 × 0.0002 ≈ 0.00001

Cela signifie que seule une occurrence sur 100,000 environ du mot "le" serait conservée pour l'entraînement !

En revanche, pour un mot comme "ordinateur" avec une fréquence de 0.0001 :
P("ordinateur") = (√(10^-5/0.0001) + 1) × (10^-5/0.0001) ≈ 0.316 × 0.1 ≈ 0.0316

Soit environ 3% des occurrences conservées, ce qui est beaucoup plus élevé que pour "le".

## Avantages du sous-échantillonnage

1. **Accélération significative de l'entraînement**
   Le nombre total d'exemples d'entraînement peut être réduit de 20 à 40%, ce qui accélère proportionnellement l'apprentissage.

2. **Amélioration de la qualité des embeddings**
   Les vecteurs résultants capturent mieux les relations sémantiques entre mots, car l'influence excessive des mots vides est réduite.

3. **Meilleure représentation des mots rares**
   L'importance relative des mots moins fréquents mais sémantiquement riches est augmentée dans l'apprentissage.

4. **Régularisation implicite**
   L'introduction d'une forme de bruit contrôlé aide à prévenir le surapprentissage sur les motifs très répétitifs.

## Stratégies d'Expérimentation


1. **Variations du Seuil**
   - Testez différentes valeurs (10^-4 à 10^-6)
   - Observez l'impact sur les performances

2. **Métriques de Comparaison**
   - Temps d'entraînement
   - Qualité des analogies (tests du type "roi - homme + femme = reine")
   - Performance en classification

3. **Visualisation**
   - Comparez les espaces vectoriels
   - Analysez les distributions de mots
